# Question 1 - Alex Wiigs Stat 553 Final

**[10 marks] A data set in "DataSet1.csv" has observations for six variable $y_1,...,y_6$ from several groups (populations). Use this data set to perform hypothesis tests and discriminant analysis.**

First we should load in the data and do some exploratory analysis. We have several different groups, so it will be important to find how many there are so we can test their poroperties.

In [17]:
library(tidyverse)

In [25]:
# Load data 
data <- read.csv("/Users/alexwiigs/Documents/Code/STAT 553 Final/DataSet1.csv")
head(round(data,2))

# shows the number of groups
num_groups <- max(data[,1])
print(paste("Number of groups in the data set:", num_groups))

# total number of entries in each group and in the whole data set
n <- nrow(data)
ni <- n / num_groups
num_groups <- 5
num_variables <- 6
sprintf("There are %f entries in the entire dataset.", n)
sprintf("There are %f entries in each group.", ni)

,group,y1,y2,y3,y4,y5,y6
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,4.71,4.28,2.12,1.60,-0.31,-0.72
2,1,4.81,3.94,2.23,-1.37,-2.02,-0.55
3,1,7.60,7.49,0.72,0.12,-0.35,-0.93
4,1,7.71,7.52,-0.05,0.38,-0.78,-0.29
5,1,6.74,5.07,1.71,1.60,1.06,-0.14
6,1,4.93,5.83,-2.62,0.74,0.25,-0.65


[1] "Number of groups in the data set: 5"


[1] "There are 650.000000 entries in the entire dataset."

[1] "There are 130.000000 entries in each group."

**(a)** Are the population covariance and matrices the same?

In [19]:
# Preliminary functions
ni <- function(i) { data |> filter(group == i) |> select(-group) |> nrow() }
Si <- function(i) { data |> filter(group == i) |> select(-group) |> cov() }
Spl <- function(data) {
    numerator = matrix(0, nrow = ncol(data[,-1]), ncol = ncol(data[,-1]))
    for (i in 1:num_groups) { numerator = numerator + ni(i) * Si(i) }
    denominator = nrow(data) - num_groups
    return(numerator / denominator)}
M <- function(data) {
    numerator = 1
    exponent = 0
    for (i in 1:num_groups) {numerator = numerator * det(Si(i))^( (ni(i) - 1) / 2 ) }
    for (i in 1:num_groups) {exponent = exponent + (ni(i) - 1) / 2}
    return( numerator / ( det(Spl(data))^exponent ) )}
c1 <- function(data) {
    term1 = 0
    term2 = 0
    for (i in 1:num_groups) {
        term1 = term1 + 1/ (ni(i) -1)
        term2 = term2 + (ni(i) -1)}
    value <- (term1 - 1 / term2) *
    (2 * num_variables^2 + 3 * num_variables - 1) / (6 * (num_variables + 1) * (num_groups - 1))
    return(value)}
# Test Statistic
u <- -2 * (1 - c1(data)) * log(M(data))
u

[1] 116.6848

**(b)** Are the population mean vectors the same?

In order to preform analysis on the different groups, it is important to first check whether the difference in the groups is statistically significant. We create a MANOVA model for this.

$$\textbf{y}_{ij} = \mu_i + \epsilon_{ij}$$
$i = 1, ..., 5$, $j = 1,..., 130$
$$\epsilon_{ij} \sim^{\text{i.i.d}} N_p(0, \Sigma )$$

By writing $\mu_i = \mu + \alpha_i$, where $\mu$ is the overall mean vector, and $\sum_{i=1}^5 \alpha_i =0$ we can state our null and alternative hypothesis as follows:  
  
$H_0: \alpha_1 = ... = \alpha_5 = 0$.     
$H_1:$ at least one $\alpha_i$ is not zero.

The Wilks' test statistic can preform this hypothesis test. The test statstic is 
$$\Lambda = \frac{|E|}{|E+H|} $$
where we will reject $H_0$ if $\Lambda \leq \Lambda_{\alpha, p, v_H, v_E}$. The matrix $E$ is a covariance matrix that represents the variance "within" the differnt groups and the matrix $H$ is a covariance matrix that repesents the variance "between" the each group and the overall average for all the groups. If $H$ is small relative to $E$, we will have little reason to reject the null hypothesis.

We calculate the "between" Matrix $H$ with the following theorem:
$$H = n \sum_{i=1}^k (\bar{\textbf{y}}_{i\cdot} - \bar{\textbf{y}}_{\cdot \cdot} )(\bar{\textbf{y}}_{i\cdot} - \bar{\textbf{y}}_{\cdot \cdot} )'$$

In [20]:
# Overall mean vector
overall_mean <- data |> select(-group) |> colSums() / ( n )
overall_mean <- overall_mean |> as.matrix()
# Calculate matrix where the rows are mean vectors for each of the five groups.
group_means <- data.frame()
for (i in 1:5) {
    temp_value <- data |> filter(group ==i) |> nrow()
    calculation <- data |> filter(group == i) |> select(-group) |> colSums() / temp_value
    group_means <- bind_rows(group_means, calculation)
    } 
group_means <- group_means |> as.matrix()
rownames(group_means) <- c("group1", "group2", "group3", "group4", "group5")

In [21]:
# Calculate the H matrix
H <- matrix(0, nrow = 6, ncol = 6)
for (i in 1:5) {
    ni <- data |> filter(group == i) |> select(-group) |> nrow()
    H <- H + ni * ( as.matrix(group_means[i,]) - overall_mean ) %*% 
    t( as.matrix(group_means[i,]) - overall_mean )  }
round(H,2)

,y1,y2,y3,y4,y5,y6
y1,25943.40,5165.60,29282.66,115.18,14.79,-3.07
y2,5165.60,9957.73,1464.52,-63.65,-44.85,67.41
y3,29282.66,1464.52,39234.27,214.80,118.35,4.02
y4,115.18,-63.65,214.80,8.62,0.77,-2.39
y5,14.79,-44.85,118.35,0.77,1.83,0.60
y6,-3.07,67.41,4.02,-2.39,0.60,1.60


Our formula for the "within" matrix $E$ is:
$$ E = \sum_{i=1}^k \sum_{j=1}^{n_i} (\textbf{y}_{ij} - \bar{ \textbf{y} }_{i \cdot} ) (\textbf{y}_{ij} - \bar{ \textbf{y} }_{i\cdot} )'$$ 

In [22]:
# Calcualte the E matrix
E <- matrix(0, nrow = 6, ncol = 6)
for (i in 1:5) {
    for ( j in 1:nrow(filter(data, group == i))) {
        y_ij <- data |> filter(group == i) |> select(-group) |> slice(j)
        E <- E + (t(y_ij) - group_means[i,] ) %*% t(t(y_ij) - group_means[i,] ) }}
round(E,2)

,y1,y2,y3,y4,y5,y6
y1,1203.34,563.79,691.20,14.37,-6.19,-4.71
y2,563.79,1304.08,-691.18,27.51,-5.94,-29.81
y3,691.20,-691.18,2020.05,-61.92,-15.48,11.81
y4,14.37,27.51,-61.92,585.53,32.29,-12.27
y5,-6.19,-5.94,-15.48,32.29,570.52,22.14
y6,-4.71,-29.81,11.81,-12.27,22.14,315.86


Now that we have calculated the matrices we can caulate the test statistic.

In [23]:
Lambda <- det(E) / det( E + H)
Lambda

[1] 0.0005536129

**(c)** Find discriminant functions and use the first two discriminant functions to show the data set.

In [24]:
vectors <- eigen(solve(E) %*% H)$vectors
values <- eigen(solve(E) %*% H )$values
